In [1]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=30318fbbf3fd482483beb5cd714b87438293d871ce4fa497031c772d422aef7d
  Stored in directory: /home/jovyan/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [10]:
import os

# 현재 작업 디렉토리 출력
print("현재 작업 디렉토리:", os.getcwd())

# 현재 디렉토리의 파일 목록 출력
print("현재 디렉토리의 파일 목록:")
for file in os.listdir():
    print(file)

# 상위 디렉토리의 파일 목록 출력
print("\n상위 디렉토리의 파일 목록:")
for file in os.listdir(".."):
    print(file)

현재 작업 디렉토리: /home/jovyan/work/learning-spark/chapter7
현재 디렉토리의 파일 목록:
iris (1).data
iris.data
.ipynb_checkpoints
test.ipynb

상위 디렉토리의 파일 목록:
chapter4.ipynb
chapter6
chapter7
java
spark-warehouse
.ipynb_checkpoints


In [14]:
import os

shared_dir = '/home/jovyan/shared'
print(f"{shared_dir} 디렉토리 정보:")
print(f"존재 여부: {os.path.exists(shared_dir)}")
if os.path.exists(shared_dir):
    print(f"권한: {oct(os.stat(shared_dir).st_mode)[-3:]}")
    print(f"소유자: {os.stat(shared_dir).st_uid}")
    print(f"그룹: {os.stat(shared_dir).st_gid}")

/home/jovyan/shared 디렉토리 정보:
존재 여부: True
권한: 755
소유자: 0
그룹: 0


In [1]:
import os
import pwd

current_user = pwd.getpwuid(os.getuid()).pw_name
print(f"현재 사용자: {current_user}")
print(f"현재 사용자 ID: {os.getuid()}")
print(f"현재 사용자 그룹 ID: {os.getgid()}")

현재 사용자: jovyan
현재 사용자 ID: 1000
현재 사용자 그룹 ID: 100


In [2]:
import os
import shutil

# 현재 작업 디렉토리 확인
current_dir = os.getcwd()
print(f"현재 작업 디렉토리: {current_dir}")

# 원본 파일 경로
source_file = os.path.join(current_dir, "iris.data")

# 대상 디렉토리 (shared 폴더로 가정)
target_dir = "/home/jovyan/shared"

# 대상 파일 경로
target_file = os.path.join(target_dir, "iris.data")

# 대상 디렉토리가 없으면 생성
os.makedirs(target_dir, exist_ok=True)

# 파일이 존재하는지 확인
if os.path.exists(source_file):
    # 파일 복사
    shutil.copy2(source_file, target_file)
    print(f"파일이 복사되었습니다: {target_file}")
else:
    print(f"원본 파일을 찾을 수 없습니다: {source_file}")

# 복사된 파일 확인
if os.path.exists(target_file):
    print(f"파일이 성공적으로 복사되었습니다.")
    print(f"복사된 파일 크기: {os.path.getsize(target_file)} bytes")
else:
    print(f"파일 복사에 실패했습니다.")

현재 작업 디렉토리: /home/jovyan/work/learning-spark/chapter7


PermissionError: [Errno 13] Permission denied: '/home/jovyan/shared/iris.data'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, StringType

# Spark 세션 생성
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:8341").\
        getOrCreate()

# 파일 경로
file_path = "/home/jovyan/work/learning-spark/chapter7/iris.data"

# 파일 내용 읽기
with open(file_path, 'r') as file:
    content = file.readlines()

# # RDD 생성
rdd = spark.sparkContext.parallelize(content)

# RDD를 DataFrame으로 변환
df = rdd.map(lambda line: line.strip().split(','))\
        .filter(lambda x: len(x) == 5)\
        .toDF(["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])

df.show()

# # DataFrame 확인
# print("DataFrame schema:")
# df.printSchema()
# print("\nFirst few rows of the DataFrame:")
# df.show(5)

# # 간단한 분석 예시
# print("\nSummary statistics:")
# df.describe().show()

# # 클래스별 개수
# print("\nCount by class:")
# df.groupBy("class").count().show()

# Spark 세션 종료
# spark.stop()

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|
|         4.8|        3.4|         1.6|        0.2|Iris-setosa|
|         4.8|        3.0|         1.4| 

In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
print(spark.sparkContext.getConf().get("spark.sql.warehouse.dir"))

file:/home/jovyan/work/learning-spark/chapter7/spark-warehouse


In [2]:
!ls -l /home/jovyan/work/learning-spark/chapter7/iris.data

-rw-r--r-- 1 jovyan users 4551 Sep 11 09:19 /home/jovyan/work/learning-spark/chapter7/iris.data


In [5]:
import os
print(os.getcwd())

/home/jovyan/work/learning-spark/chapter7


In [8]:
import os
print(os.path.exists('/home/jovyan/work/learning-spark/chapter7/iris.data'))

True


In [15]:
spark.sparkContext.addFile("/home/jovyan/work/learning-spark/chapter7/iris.data")
data = spark.read.csv("file://" + SparkFiles.get("iris.data"))
# data = spark.read.csv("/home/jovyan/work/learning-spark/chapter7/iris.data")
# data = spark.read.csv('/home/jovyan/work/learning-spark/chapter7/iris.data')
# data.show(n=5)

NameError: name 'SparkFiles' is not defined